In [12]:
from sklearn.model_selection import train_test_split
import torch

In [13]:
racist_x_train = torch.tensor(train_data_unintended_bias.iloc[:, 8:32].values, dtype = torch.float)
racist_y_train = torch.tensor(train_data_unintended_bias.iloc[:, 2], dtype = torch.float)

# racist_y_train = torch.tensor(racist_y_train.apply(lambda x: int(x)).values)
# print(racist_y_train[0])

racist_x_train, racist_x_test, racist_y_train, racist_y_test = train_test_split(
    racist_x_train, 
    racist_y_train, 
    test_size = 0.3)

In [14]:
class CatDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        super().__init__()
        
        self.x = x
        self.y = y
        self.index_list = []
        
    def __len__(self):
        return self.x.shape[0]
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]

In [15]:
import torch.nn as nn

In [16]:
class RacistClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear1 = nn.Linear(24, 32)
        self.softmax = nn.Softmax(dim = 1)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(32, 1)
    
    def forward(self, x):
        return self.softmax(self.linear2(self.relu(self.linear1(x.float()))))

In [17]:
model = RacistClassifier()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)
mse = nn.MSELoss()

In [18]:
training_set = CatDataset(racist_x_train, racist_y_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size = 128)

validation_set = CatDataset(racist_x_test, racist_y_test)
validation_generator = torch.utils.data.DataLoader(validation_set)

n_iterations = 100

from sklearn.metrics import accuracy_score
from tqdm import tqdm

loss = 0

# Loop over epochs
for epoch in tqdm(range(n_iterations)):
    # Training
    for local_batch, local_labels in training_generator:
        optimizer.zero_grad()
        # Get predictive output
        output = model(local_batch)
    #     for i in tqdm(range(racist_y_train.shape[0])):
    #         if racist_y_train[i] != 0 or racist_y_train[i] != 1:
    #             print(racist_y_train[i])
        # Calc loss and backprop gradients
        loss = mse(output.reshape(-1), local_labels)
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(loss)

#     print(loss)
    # Validation
#     with torch.set_grad_enabled(False):
#         for local_batch, local_labels in validation_generator:
#             print(accuracy_score(model(local_batch).round().detach().numpy(), local_labels.round()))
#             break

  0%|          | 0/100 [00:00<?, ?it/s]

tensor(0.8565, grad_fn=<MseLossBackward>)
tensor(0.8528, grad_fn=<MseLossBackward>)
tensor(0.8460, grad_fn=<MseLossBackward>)
tensor(0.7958, grad_fn=<MseLossBackward>)
tensor(0.8622, grad_fn=<MseLossBackward>)
tensor(0.8520, grad_fn=<MseLossBackward>)
tensor(0.8324, grad_fn=<MseLossBackward>)
tensor(0.8270, grad_fn=<MseLossBackward>)
tensor(0.8297, grad_fn=<MseLossBackward>)
tensor(0.7977, grad_fn=<MseLossBackward>)
tensor(0.8482, grad_fn=<MseLossBackward>)
tensor(0.8531, grad_fn=<MseLossBackward>)
tensor(0.8723, grad_fn=<MseLossBackward>)
tensor(0.8475, grad_fn=<MseLossBackward>)
tensor(0.8425, grad_fn=<MseLossBackward>)
tensor(0.8118, grad_fn=<MseLossBackward>)
tensor(0.8699, grad_fn=<MseLossBackward>)
tensor(0.8332, grad_fn=<MseLossBackward>)
tensor(0.8287, grad_fn=<MseLossBackward>)
tensor(0.8088, grad_fn=<MseLossBackward>)
tensor(0.8567, grad_fn=<MseLossBackward>)
tensor(0.8138, grad_fn=<MseLossBackward>)
tensor(0.8422, grad_fn=<MseLossBackward>)
tensor(0.8405, grad_fn=<MseLossBac

  1%|          | 1/100 [00:28<46:38, 28.27s/it]


tensor(0.8608, grad_fn=<MseLossBackward>)
tensor(0.8390, grad_fn=<MseLossBackward>)
tensor(0.8333, grad_fn=<MseLossBackward>)
tensor(0.8494, grad_fn=<MseLossBackward>)
tensor(0.8295, grad_fn=<MseLossBackward>)
tensor(0.8469, grad_fn=<MseLossBackward>)
tensor(0.8585, grad_fn=<MseLossBackward>)
tensor(0.8408, grad_fn=<MseLossBackward>)
tensor(0.8469, grad_fn=<MseLossBackward>)
tensor(0.8544, grad_fn=<MseLossBackward>)
tensor(0.8490, grad_fn=<MseLossBackward>)
tensor(0.8230, grad_fn=<MseLossBackward>)
tensor(0.8380, grad_fn=<MseLossBackward>)
tensor(0.8267, grad_fn=<MseLossBackward>)
tensor(0.8794, grad_fn=<MseLossBackward>)
tensor(0.8675, grad_fn=<MseLossBackward>)
tensor(0.8406, grad_fn=<MseLossBackward>)
tensor(0.8338, grad_fn=<MseLossBackward>)
tensor(0.8635, grad_fn=<MseLossBackward>)
tensor(0.8192, grad_fn=<MseLossBackward>)
tensor(0.8657, grad_fn=<MseLossBackward>)
tensor(0.8185, grad_fn=<MseLossBackward>)
tensor(0.8492, grad_fn=<MseLossBackward>)
tensor(0.8505, grad_fn=<MseLossBa

  3%|▎         | 3/100 [01:02<36:10, 22.38s/it]

KeyboardInterrupt: 

In [ ]:
# from tqdm import tqdm


# for i, x_train, y_train in tqdm(zip(range(n_iterations), racist_x_train, racist_y_train)):
#     print(i)
#     # Zero backpropped gradients from previous iteration
#     optimizer.zero_grad()
#     # Get predictive output
#     output = model(racist_x_train)
# #     for i in tqdm(range(racist_y_train.shape[0])):
# #         if racist_y_train[i] != 0 or racist_y_train[i] != 1:
# #             print(racist_y_train[i])
#     # Calc loss and backprop gradients
#     loss = mse(output.reshape(-1), racist_y_train)
#     loss.backward()
#     optimizer.step()

In [ ]:
# from sklearn.metrics import accuracy_score
# print(model(racist_x_test).round())
accuracy_score(model(racist_x_test).round().detach().numpy(), racist_y_test.round())

In [ ]:
model(racist_x_test)

In [ ]:
racist_y_test.round()